In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

Batch = 192
sizeTest = 64
SizeTrain = Batch - sizeTest

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

trainset = datasets.MNIST('TrainSet', download=True, train=True, transform=transform)
valset = datasets.MNIST('ValSet', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=Batch, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.numpy()
images = np.resize(images,(Batch,784))
X = images[0:SizeTrain,:]
Y = labels.numpy()[0:SizeTrain]


new_column = np.ones(SizeTrain)
X = np.column_stack([X, new_column])

Xval = images[SizeTrain:Batch,:]
Yval = labels[SizeTrain:Batch]

new_column = np.ones(sizeTest)
Xval = np.column_stack([Xval, new_column])

In [29]:
newY = np.zeros(shape=(SizeTrain,10))
for i in range(Y.shape[0]):
    newY[i,Y[i]] = 1


In [ ]:
import numpy as np

# Number of classes
K = 10

# Number of features
N = 785

# Initialize weights
weights = np.zeros((K,N))

#Training Iterations
trainIters = 1000

# sigmoid
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# Training method
def one_vs_all_logreg(X, y, K, weights, learning_rate, lamb):
    
    # Number of training examples
    m = X.shape[0]
    
    for i in range(trainIters):
    # Initialize the cost
      J = 0
      
      # Compute the hypothesis
      h = np.dot(X, weights.T)
      
      # Compute the cost
      for k in range(K):
          temp = -y[:,k] * np.log(sigmoid(h[:,k])) - (1 - y[:,k]) * np.log(1 - sigmoid(h[:,k]))
          J += np.sum(temp)
          g = np.dot(X.T, (sigmoid(h)[:,k] - y[:,k]))
          g = g/m
          g = g + (lamb/m * np.max(weights[1:,k]))
          

          #update weights 
          weights[k] = weights[k] - learning_rate*g
          
      J = J/m

    return weights

      
    


theta = one_vs_all_logreg(X, newY, K, weights, .1)

In [30]:
def predict(X, weights):
    return np.argmax(sigmoid(np.dot(X, weights.T)), axis=1)

pred = predict(Xval,theta)


In [31]:
numFalse = 0
for i in range(pred.shape[0]):
    if pred[i] == Yval[i]:
        numFalse = numFalse
    else:
        numFalse = numFalse + 1
print(numFalse/sizeTest)

0.890625
